In [2]:
#import the Libraries 
import numpy as nu #used for data structure
import matplotlib.pyplot as plt #visualisation 

**Equation to be used:**      
$f(n) = {236.n^{2.54} + n.m_n - \rho_s}$       
$f'(n) = 236 \times 2.54 \times n^{1.54} + m_n$

**Newton Raphson Method:**          
$n_{i+1} = n_i - \frac{f(n_i)}{f'(n_i)}$

In [3]:
def initial_n():
    n = 1 # initial value of n
    err = 1 # initial Error
    tol = 1e-15 # Tolerance 
    count = 0 # total iteration 

    #Newton Raphson Method
    with err > tol:
        count+=1
        fn = 236*n**(2.54) + n*mn - rho_s # Find f(n_old)
        dfn = 236*2.54*n**(1.54) + mn
        temp = n - fn/dfn # n_new
        err = np.abs(n-temp)
        n = temp
    print("Newton-Raphson Converged after ", count, "iterations")
    return n

In [4]:
def rho(p):

    '''
    Energy Density of a neutron star at a given pressure
    rho_s - Central Density at r = 0
    mn - mass of a neutron
    n - number density at given pressure
    '''

    n = (p*rho_s/363.44)**(1/2.54)
    return (236. * n**2.54 + n *mn)/rho_s